[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=RawTransactions.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=RawTransactions.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Raw Transactions

`mango-explorer` uses `CombinableInstructions`  throughout. This might be a problem if you're trying to integrate with other systems that use Solana's own `Transaction`s and `Instruction`s.

This example is nearly identical to the [CombinableInstructions](CombinalbleInstructions.ipynb) example, except that the execution of the place/crank/settle instructions uses raw `Transaction` and `TransactionInstruction` objects instead of `CombinableInstructions`' `execute()` method.

## Identical Prologue

This section of code is identical to the [CombinableInstructions](CombinalbleInstructions.ipynb) example.

In [ ]:
import decimal
import mango

# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = mango.Wallet(bytes([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146]))

# Signers are effectively an empty CombinableInstruction that only carries the keys for
# signing transactions
signers: mango.CombinableInstructions = mango.CombinableInstructions.from_wallet(wallet)

# Create a 'devnet' Context
context = mango.ContextBuilder.build(cluster_name="devnet")
# Load the wallet's account
group = mango.Group.load(context)
accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
account = accounts[0]

# Create the right MarketOperations from the Market. We use this as an easy way to display orders.
market_operations = mango.operations(context, wallet, account, "SOL-PERP", dry_run=False)

print("Orders (initial):")
print(market_operations.load_orderbook())

# Create the right MarketInstructionBuilder from the Market
market_instructions = mango.instruction_builder(context, wallet, account, "SOL-PERP", dry_run=False)

# Go on - try to buy 1 SOL for $10.
client_id = context.generate_client_id()
order = mango.Order.from_values(side=mango.Side.BUY,
                                price=decimal.Decimal(10),
                                quantity=decimal.Decimal(1),
                                order_type=mango.OrderType.POST_ONLY).with_update(client_id=client_id)


## Raw Transaction And TransactionInstructions

This is the section of code that is different to the [CombinableInstructions](CombinalbleInstructions.ipynb) example.

Note that `market_instructions` is still used to build `CombinableInstructions` - it's the native way `mango-explorer` works - but they're then 'unwrapped'.

`CombinableInstructions` maintains a list of `TransactionInstruction`s in its `instructions` property. You can just add them to a `Transaction`s instructions by calling `extend()` on the transaction's `instructions`, for example:
```
tx.instructions.extend(combinable_instruction.instructions)
```
Signing the transaction can be done by extracting the signers from the `CombinableInstructions`' `signers` property:
```
signers = []
signers.extend(combinable_instruction.signers)
...
context.client.send_transaction(tx, *signers)
```
In the common case, where there's only the wallet signer, you can just do:
```
context.client.send_transaction(tx, wallet.keypair)
```

In [ ]:
print("\n\nPlacing order:\n", order)

# Build the individual CombinableInstructions. You could add others here - the marketmaker adds in
# order cancellations and optional MNGO redeem instructions.
place_order = market_instructions.build_place_order_instructions(order)
crank = market_instructions.build_crank_instructions([])
settle = market_instructions.build_settle_instructions()

# Instead of combining them like this:
# (signers + place_order + crank + settle).execute(context)
# We now need to build the Transaction and add the Instructions and signers manually.

# Import here so the above code block really is identical to the CombinableInstructions example.
from solana.transaction import Transaction

transaction = Transaction()
transaction.instructions.extend(place_order.instructions)
transaction.instructions.extend(crank.instructions)
transaction.instructions.extend(settle.instructions)

print("Sending transaction:", transaction)
transaction_signature = context.client.send_transaction(transaction, wallet.keypair)
place_signatures = [transaction_signature]


## Identical Epilogue

This section is also identical to the [CombinableInstructions](CombinalbleInstructions.ipynb) example.

In [ ]:

print("Waiting for place order transaction to confirm...\n")
mango.WebSocketTransactionMonitor.wait_for_all(
        context.client.cluster_ws_url, place_signatures, commitment="processed"
    )

print("\n\nOrders (including our new order):")
print(market_operations.load_orderbook())

cancel_signatures = market_operations.cancel_order(order)
print("\n\ncancel_signatures:\n\t", cancel_signatures)

print("Waiting for cancel order transaction to confirm...\n")
mango.WebSocketTransactionMonitor.wait_for_all(
        context.client.cluster_ws_url, cancel_signatures, commitment="processed"
    )

print("\n\nOrders (without our order):")
print(market_operations.load_orderbook())

context.dispose()
print("Example complete.")